In [0]:
# =====================================================
# NOTEBOOK : Contrôle qualité des données météo
# ETAPE : Data Quality Check
# PROJET : Big Data - Spark & Climat Sénégal
# =====================================================

« Les données météorologiques sont stockées dans une table Delta afin d’assurer la persistance, la réutilisabilité inter-notebooks et la reproductibilité du pipeline Big Data. »

In [0]:
# Rechargeons les données depuis la table weather_senegal_raw
df_weather = spark.read.table("weather_senegal_raw")

In [0]:
# On retient les colonnes avec les valeurs non nulles
VALID_COLUMNS = [
    "time", "tavg", "tmin", "tmax",
    "prcp", "wspd", "pres", "region"
]

df_qc = df_weather.select(VALID_COLUMNS)

df_qc.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- tavg: double (nullable = true)
 |-- tmin: double (nullable = true)
 |-- tmax: double (nullable = true)
 |-- prcp: double (nullable = true)
 |-- wspd: double (nullable = true)
 |-- pres: double (nullable = true)
 |-- region: string (nullable = true)



In [0]:
# On détermine ici les valeurs manquantes par variable
from pyspark.sql.functions import col, sum as spark_sum

missing_count = df_qc.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in df_qc.columns
])

display(missing_count)

time,tavg,tmin,tmax,prcp,wspd,pres,region
0,10965,22462,12724,44370,40648,48188,0


In [0]:
# On détermine le pourcentage des valeurs manquantes
total_rows = df_qc.count()
missing_percent = missing_count.select([
    (col(c) / total_rows * 100).alias(c)
    for c in missing_count.columns
])

display(missing_percent)

time,tavg,tmin,tmax,prcp,wspd,pres,region
0.0,16.681626629748518,34.17261261809496,19.357685110526234,67.50239612968006,61.83992332385024,73.31091874458018,0.0


In [0]:
# On détermine la courverture géographique des données
coverage_region = (
    df_qc
    .groupBy("region")
    .count()
    .orderBy("region")
)

display(coverage_region)


region,count
Dakar,7306
Diourbel,7305
Kaolack,7303
Kolda,7303
Kédougou,7303
Matam,7303
Saint-Louis,7302
Tambacounda,7303
Ziguinchor,7303


In [0]:
# On détermine deux colonnes pour déterminer les dates de début et de fin de période pour chaque région.
from pyspark.sql.functions import min, max

time_coverage = (
    df_qc
    .groupBy("region")
    .agg(
        min("time").alias("date_min"),
        max("time").alias("date_max")
    )
)

display(time_coverage)

region,date_min,date_max
Dakar,2005-01-01T00:00:00.000Z,2025-01-01T00:00:00.000Z
Diourbel,2005-01-02T00:00:00.000Z,2025-01-01T00:00:00.000Z
Ziguinchor,2005-01-04T00:00:00.000Z,2025-01-01T00:00:00.000Z
Tambacounda,2005-01-04T00:00:00.000Z,2025-01-01T00:00:00.000Z
Saint-Louis,2005-01-05T00:00:00.000Z,2025-01-01T00:00:00.000Z
Matam,2005-01-04T00:00:00.000Z,2025-01-01T00:00:00.000Z
Kolda,2005-01-04T00:00:00.000Z,2025-01-01T00:00:00.000Z
Kédougou,2005-01-04T00:00:00.000Z,2025-01-01T00:00:00.000Z
Kaolack,2005-01-04T00:00:00.000Z,2025-01-01T00:00:00.000Z


Les variables météorologiques effectivement disponibles ont été identifiées avant le contrôle qualité afin d’éviter l’inclusion de colonnes structurellement vides. Le contrôle qualité s’est focalisé sur les variables réellement observées. On passe à présent au nettoyage des données météorologiques dans le notebook 04.Data_clean_weather.